#### ENTER ACCESS_KEY AND SECRET KEY

In [10]:
Access_Key = "LMWLG31WZS"
Secret_Key = "jxAvgFWSoZmFdlTpmKCv"

#### ENTER PIPELINE NUMBER, SOURCE PASSWORD, DESTINATION PASSWORD

In [11]:
pipeline_number = 1314
source_password = 'root'
destination_password = 'demoDEMO1236'
Region = 'US'
database_name = 'Ecommerce' 

### Get Pipeline API call

In [12]:
import requests
import base64

  
sample_string = (Access_Key)+":"+(Secret_Key)
sample_string_bytes = sample_string.encode("ascii")
  
base64_bytes = base64.b64encode(sample_string_bytes)
base64_string = base64_bytes.decode("ascii")
  
#print(f"Encoded string: {base64_string}")

url = 'https://'+ Region + '.hevodata.com/api/public/v2.0/pipelines/'+ str(pipeline_number)
headers = {'Accept':'application/json','Authorization': 'Basic '+ base64_string}
r = requests.get(url, headers=headers)

#print (r.json())
#print (r.json()['data']['destination'])

### Initialize Source connection

In [53]:
# import great_expectations as ge
#import psycopg2 
import pymysql
import mysql.connector as mysql
import pandas as pd

#switch case
user = r.json()['data']['source']['config']['db_user']
password = r.json()['data']['source']['config']['db_password']
port = r.json()['data']['source']['config']['db_port']
host = r.json()['data']['source']['config']['db_host']
db = r.json()['data']['source']['config']['db_names']
if r.json()['data']['source']['type']['name']=='MYSQL':
    CONN=mysql.connect(host=str(host),user=str(user),passwd='root',db=database_name) #case if many dbs are connected
elif r.json()['data']['source']['type']['name']=='POSTGRES':
    CONN=psycopg2.connect(host=host,port=port,user=user,password='root',database=db)

# df = pd.read_sql_query('SELECT COUNT(distinct ID) as rows FROM Ecommerce.Users ',CONN)
# df.head(1000)

,rows
0,513


### Initialize Destination connection (Redshift,Snowflake,MySQL,Postgres)

In [14]:
import snowflake.connector
import redshift_connector
import pandas as pd

if r.json()['data']['destination']['type']['name'] == 'SNOWFLAKE':
    db_name = r.json()['data']['destination']['config']['db_name']
    db_user = r.json()['data']['destination']['config']['db_user']
    account_name = r.json()['data']['destination']['config']['account_name']
    schema_name = r.json()['data']['destination']['config']['schema_name']
    warehouse = r.json()['data']['destination']['config']['warehouse']
    region = r.json()['data']['destination']['config']['region']
    conn =  snowflake.connector.connect(user=str(db_user), password=str(destination_password), 
                                                    account=str(account_name) + "." + str(region), role='ACCOUNTADMIN', warehouse=warehouse,
                                                    database = str(db_name), schema = str(schema_name))
elif r.json()['data']['destination']['type']['name'] == 'REDSHIFT':    
    db_name = r.json()['data']['destination']['config']['db_name']
    db_host = r.json()['data']['destination']['config']['db_host']
    db_user = r.json()['data']['destination']['config']['db_user']
    db_port = r.json()['data']['destination']['config']['db_port']
    conn = redshift_connector.connect(host=str(db_host),user=str(db_user),password='demoDEMO1236'
                                 ,port=db_port,database=str(db_name))
elif r.json()['data']['destination']['type']['name'] == 'MYSQL' or 'POSTGRES':
    user = r.json()['data']['source']['config']['db_user']
    password = r.json()['data']['source']['config']['db_password']
    port = r.json()['data']['source']['config']['db_port']
    host = r.json()['data']['source']['config']['db_host']
    db = r.json()['data']['source']['config']['db_names']
    if r.json()['data']['destination']['type']['name'] == 'MYSQL':
        CONN=mysql.connect(host=str(host),user=str(user),passwd='root',db=database_name)
    elif r.json()['data']['destination']['type']['name'] == 'POSTGRES':
        CONN=psycopg2.connect(host=host,port=port,user=user,password='root',database=db)
        
        
#conn
# dff = pd.read_sql_query("Select * from demo.public.mysql_data_ecommerce_products",conn)
# dff.head(20)

### Loop through source tables

In [15]:
url = "https://us.hevodata.com/api/public/v2.0/pipelines/"+str(pipeline_number)+"/event-types?limit=100"

headers = {
    "Accept": "application/json",
    "Authorization": 'Basic '+ base64_string
}

response = requests.request("GET", url, headers=headers)

#print(response.json())
table_count = (response.json()['pagination']['count'])
source_tables = []
for i in range(table_count):
    source_tables.append(response.json()['data'][i]['event_type'])
    
print(source_tables)

['Products', 'Users', 'userAddress']


### Source Row Count

In [43]:
#df = pd.read_sql_query('show tables',db)
source_row_count={}
for table in source_tables:
        #print (column)
    source_row_count.setdefault(table,0)
#print(source_row_count)

for i in source_row_count.keys():
    df = pd.read_sql_query('SELECT COUNT(*) as rows FROM Ecommerce.'+i,CONN)
    source_row_count[i]=df.rows[0] 
#source_row_count
row_counts_1 = []
for i in source_row_count.values():
    row_counts_1.append(i)
row_counts_1


{'Products': 504, 'Users': 513, 'userAddress': 505}

### Loop through respective destination tables

In [44]:
#Fetch Prefix??
destination_tables=[]
for i in source_tables:
    url = "https://us.hevodata.com/api/public/v2.0/pipelines/"+str(pipeline_number)+"/mappings/" + i

    headers = {
        "Accept": "application/json",
        "Authorization": 'Basic '+ base64_string
    }

    response = requests.request("GET", url, headers=headers)
    destination_tables.append(response.json()['data']['destination_table'])
#     print(response.json()['data'])
destination_tables

KeyError: 'destination_table'

### Destination Row Count

In [ ]:
destination_row_count={}
for table in destination_tables:
        #print (column)
    destination_row_count.setdefault(table,0)
#print(destination_row_count)
for i in destination_row_count.keys():
    dff = pd.read_sql_query("Select COUNT(*) as rows from demo.public."+i,conn)
    destination_row_count[i]=dff.rows[0] 
#     print (dff.head())
#destinaton_row_count = {k.lower(): v for k,v in destination_tables.items()}
print(destination_row_count)
row_counts_2=[]
for i in destination_row_count.values():
    row_counts_2.append(i)
#row_counts_1 = source_row_count.values()
row_counts_2
# dff = pd.read_sql_query("Select * from demo.public.test1_users",conn)
# dff.head()

## Plotting

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.arange(len(source_tables))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, row_counts_1, width, label='Source')
rects2 = ax.bar(x + width/2, row_counts_2, width, label='Destination')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Count')
ax.set_title('Table count in source and destination')
ax.set_xticks(x) # values
ax.set_xticklabels(source_tables) # labels
ax.legend()

# ax.bar_label(rects1, padding=3)
# ax.bar_label(rects2, padding=3)

fig.tight_layout()

plt.show()

### Compare Row Count of both tables

In [ ]:
# data.expect_table_row_count_to_equal(df.shape[0])

### Compare Column Count

In [ ]:
# data.expect_table_column_count_to_equal(df.shape[1])

### Column Values to not be NULL

In [ ]:
# data.expect_column_values_to_not_be_null('ID')

### Column values to be unique

In [ ]:
# data.expect_column_values_to_be_unique('ID')

In [ ]:
1. Change source and dest count queries - N done
2. Cluster input - N done
3. Requirements.txt - S
4. DataFrame output - S
5. Source : add MS_SQL, MongoDB - N
6. Destination: add MySQL, PG - N done 
7. Clean code - N 
8. DB name input - N - done
9. Unique count - N - 'SELECT ID, COUNT(*) FROM Ecommerce.Users GROUP BY 1 HAVING COUNT(*) = 1',CONN  
'SELECT COUNT(distinct ID) as rows FROM Ecommerce.Users' done
10.Output to excel - S

In [4]:
import pyodbc 

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=http://solutions-demodb.hevo.me/;'
                      'Database=ecommerce;'
                      'Trusted_Connection=yes;'
                      'UID=root;'
                      'PWD=root;')

df = pd.read_sql_query('SELECT * FROM products', conn)

print(df)
print(type(df))

Error: ('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'SQL Server' : file not found (0) (SQLDriverConnect)")

In [6]:
pyodbc.drivers()

[]